# シャープ化フィルタ

このノートブックでは、シャープ化フィルタについて学びます。

## 目次
1. [ラプラシアンフィルタ](#ラプラシアンフィルタ)
2. [アンシャープマスキング](#アンシャープマスキング)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from skimage import filters, feature

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["axes.unicode_minus"] = False
np.random.seed(42)
print("ライブラリのインポートが完了しました。")

## ラプラシアンフィルタ

In [ ]:
# ラプラシアンフィルタによるシャープ化
# エッジを強調して画像をシャープにします

# サンプル画像の作成
img = np.zeros((200, 200), dtype=np.uint8)
img[50:150, 50:150] = 200
img = cv2.GaussianBlur(img, (5, 5), 0)  # 少しぼかす

# ラプラシアンフィルタの適用
laplacian = cv2.Laplacian(img, cv2.CV_64F)
laplacian_abs = np.uint8(np.absolute(laplacian))

# シャープ化: 元の画像 + ラプラシアン
sharpened = np.clip(img.astype(np.float32) + laplacian_abs.astype(np.float32), 0, 255).astype(np.uint8)

# より強くシャープ化
sharpened_strong = np.clip(img.astype(np.float32) + 2 * laplacian_abs.astype(np.float32), 0, 255).astype(np.uint8)

# 結果の表示
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('元の画像（ぼかし）')
axes[0, 0].axis('off')

axes[0, 1].imshow(laplacian_abs, cmap='gray')
axes[0, 1].set_title('ラプラシアン')
axes[0, 1].axis('off')

axes[1, 0].imshow(sharpened, cmap='gray')
axes[1, 0].set_title('シャープ化（通常）')
axes[1, 0].axis('off')

axes[1, 1].imshow(sharpened_strong, cmap='gray')
axes[1, 1].set_title('シャープ化（強）')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print("ラプラシアンフィルタによるシャープ化:")
print("- エッジを強調")
print("- 画像 = 元の画像 + α × ラプラシアン")
print("- αが大きいほど強くシャープ化")

## アンシャープマスキング

In [ ]:
# アンシャープマスキング（Unsharp Masking）
# より自然なシャープ化手法

# 元の画像
img = np.zeros((200, 200), dtype=np.uint8)
img[50:150, 50:150] = 200
img = cv2.GaussianBlur(img, (5, 5), 0)

# アンシャープマスキング
# 1. 画像をぼかす
blurred = cv2.GaussianBlur(img, (5, 5), 0)

# 2. マスクを作成（元の画像 - ぼかした画像）
mask = img.astype(np.float32) - blurred.astype(np.float32)

# 3. シャープ化（元の画像 + α × マスク）
alpha = 1.5
sharpened_unsharp = np.clip(img.astype(np.float32) + alpha * mask, 0, 255).astype(np.uint8)

# 異なる強度で比較
sharpened_weak = np.clip(img.astype(np.float32) + 0.5 * mask, 0, 255).astype(np.uint8)
sharpened_strong = np.clip(img.astype(np.float32) + 2.0 * mask, 0, 255).astype(np.uint8)

# 結果の表示
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('元の画像')
axes[0, 0].axis('off')

axes[0, 1].imshow(blurred, cmap='gray')
axes[0, 1].set_title('ぼかした画像')
axes[0, 1].axis('off')

axes[0, 2].imshow(mask, cmap='gray')
axes[0, 2].set_title('マスク（元 - ぼかし）')
axes[0, 2].axis('off')

axes[1, 0].imshow(sharpened_weak, cmap='gray')
axes[1, 0].set_title('アンシャープ（弱）')
axes[1, 0].axis('off')

axes[1, 1].imshow(sharpened_unsharp, cmap='gray')
axes[1, 1].set_title('アンシャープ（通常）')
axes[1, 1].axis('off')

axes[1, 2].imshow(sharpened_strong, cmap='gray')
axes[1, 2].set_title('アンシャープ（強）')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("アンシャープマスキング:")
print("- より自然なシャープ化")
print("- 手順: 1) ぼかす 2) マスク作成 3) 元の画像 + α × マスク")
print("- ラプラシアンより自然な結果")